# Baseline - Most popular
Most popular books are the top most rated books

In [121]:
from bibrec.server.Utils import assign_popular_based_score
import pandas as pd
import numpy as np
import bibrec.server.evaluation as eval
from collections import defaultdict
import bibrec.server.Utils as Utils
import importlib
from sklearn.model_selection import train_test_split

importlib.reload(eval)
importlib.reload(Utils)

books, users, ratings = Utils.get_normalized_data()

/Users/johannes/Studium/MMT/1. Semester/Fächer/Recommender Systems/Abschlussprojekt/code/recommender-systems/bibrec/server/Utils.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=";", encoding="latin-1")
/Users/johannes/Studium/MMT/1. Semester/Fächer/Recommender Systems/Abschlussprojekt/code/recommender-systems/bibrec/server/Utils.py:98: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)


Normalized Ratings Count: 383962


## Splitting the Data
In [the Notebook File baseline3](notebooks/baseline3.ipynb), the splitting into training/testing data was done on a per-user basis.
It was found that this only minimally increases the recall and precision of the baseline.
Due to performance reasons, we reverted back to a normal train/test split.
normal splitting since user specific splitting doesn't make that much of a difference and takes a long time


In [149]:
train, test = train_test_split(ratings, test_size=0.25)

**The split was also tested with 0.2 and 0.3**
- 0.2 - Precision 0.00133 and Recall 0.02313
- 0.3 - Precision 0.00149 and Recall 0.0219

In [150]:
print("Total Ratings Count:", len(ratings))
print("Train Data Ratings Count:", len(train))
print("Test Data Ratings Count:", len(test))

Total Ratings Count: 383962
Train Data Ratings Count: 287971
Test Data Ratings Count: 95991


In [151]:
book_ratings_dict = {
    'user_id': train['user_id'].values,
    'isbn13': train['isbn13'].values,
    'book_rating': train['book_rating'].values
}

book_rating_df = train[['user_id', 'isbn13', 'book_rating']]

In [152]:
train["isbn13"] = train["isbn13"].astype("int")
books["isbn13"] = books["isbn13"].astype("int")
popular_books = assign_popular_based_score(train, books, "user_id", "isbn13", "book_rating")
popular_books = popular_books.sort_values("vote_count", ascending=False)

print("Most popular books are:")
print(popular_books[:20])

Most popular books are:
              isbn13  vote_count  avg_rating  weighted_rating
9169   9780316666343         545    8.174312         8.171968
33511  9780971880108         441    4.414966         4.429044
14316  9780385504201         368    8.375000         8.370450
8036   9780312195519         287    8.177700         8.173241
4652   9780060928339         248    7.814516         7.812266
23536  9780590353427         224    8.919643         8.907374
7088   9780142001745         223    8.434978         8.426963
1757    978044667227         216    8.083333         8.078287
20395  9780452282155         212    7.985849         7.981620
9150   9780316601955         210    7.561905         7.561635
24734  9780671027360         207    8.120773         8.115151
17047  9780440237228         193    7.378238         7.379829
26815  9780679764021         191    7.701571         7.699827
29813  9780786868711         187    7.909091         7.905114
8212   9780312278588         174    7.655172  

In [153]:
pop_dict = {
    'item_id': popular_books['isbn13'].values,
    'est_r': popular_books['avg_rating'].values,
}

pop_df = pd.DataFrame.from_dict(pop_dict)

ratings_dict = {
    'user_id': test['user_id'].values,
    'item_id': test['isbn13'].values,
    'rating': test['book_rating'].values
}

ratings_df = pd.DataFrame.from_dict(ratings_dict)

In [154]:
test_uids = ratings_df["user_id"].unique()

print("User Count:", len(test_uids))

User Count: 29838


In [155]:
def def_value():
    return "Not Present"

top_n = defaultdict(def_value)
for uid in test_uids:
    top_n[uid] = pop_df

In [166]:
import time

start_time = time.time()
avg_precision = eval.get_avg_precision(ratings_df, top_n, k=50)

print("Average Precision:", avg_precision)
print("--- Calculation time: %s seconds ---" % (time.time() - start_time))

Average Precision: 0.0009564055357524559
--- Calculation time: 60.37415409088135 seconds ---


In [167]:
avg_recall = eval.get_avg_recall(ratings_df, top_n, k=50)

print("Average Recall:", avg_recall)
print("--- Calculation time: %s seconds ---" % (time.time() - start_time))

Average Recall: 0.05060547583164393
--- Calculation time: 126.55736088752747 seconds ---


**Precision and Recall has also been calculated with different k's:**
- k=25: Precision 0.00169 and Recall 0.01567
- k=50: Precision 0.00143 and Recall 0.02329
- k=75: Precision 0.00124 and Recall 0.02944
- k=100: Precision 0.00124 and Recall 0.03518
- k=200: Precision 0.00096 and Recall 0.05061